In [1]:
import sys
import os
build_path = os.path.join(os.getcwd(), "..", "build")

In [ ]:
import psutil
!cmake .. -B {build_path} -DPYTHON_EXECUTABLE={sys.executable} -Wno-dev -DCMAKE_BUILD_TYPE=Release
!cmake --build {build_path} -t pypumila -j {psutil.cpu_count()}

In [3]:
os.chdir(build_path)
import pumila
# to reload pypumila, restart python kernel.
# importlib.reload(pypumila) <- does not work

import numpy as np
from tqdm.notebook import tqdm
import random
import matplotlib
import matplotlib.pyplot as plt
import torch

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
learning = pumila.Learning(Net=pumila.Net14)
print(learning.device)

mps


In [ ]:
num_step = 100000
reset_step = 100
sim = pumila.GameSim()
steps = []
for i_episode in tqdm(range(num_step)):
    step, feat = learning.get_step(sim)
    steps.append(step)
    action = learning.select_action(feat)
    learning.memory.push(step, feat, action)
    sim.put(pumila.actions[action])
    sim.step()
    assert sim.phase_get() != pumila.GameSim.PhaseEnum.free
    while sim.phase_get() != pumila.GameSim.PhaseEnum.free:
        sim.step()
    learning.optimize_model()
    learning.update_target()
    if sim.is_over or sim.step_count >= reset_step:
        sim.reset()

In [ ]:
torch.save(learning.policy_net, "pumila14.pt")

In [13]:
win = pumila.Window()
sim = pumila.GameSim()
win.set_sim([(sim, False)])

In [14]:
import time
while True:
    time.sleep(0.016)
    if sim.phase_get() == pumila.GameSim.PhaseEnum.free:
        step = sim.current_step()
        feat = torch.from_numpy(policy_net.calc_action(step)).to(torch.float32).to(device)
        action = select_action(feat)
        sim.soft_put(pumila.actions[action])
    sim.step()
    win.step()

IndexError: array::at

In [16]:
sim.reset()

AttributeError: 'pypumila.GameSim' object has no attribute 'reset'